# Lesson 4: Aggregate Functions and GROUP BY

**Duration:** 20 minutes  
**Prerequisites:** Complete Lessons 1-3  
**Learning Mode:** Read explanations, then run each SQL query

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
- Use aggregate functions (COUNT, AVG, MAX, MIN, SUM)
- Group data with GROUP BY
- Filter groups with HAVING
- Understand the difference between WHERE and HAVING
- Calculate statistics on your data


## 📚 What are Aggregate Functions?

**Aggregate functions** perform calculations on multiple rows and return a single result. They help you answer questions like:

- How many characters are in the database?
- What's the average height?
- Who is the tallest character?
- How many characters of each species are there?

### The Five Main Aggregate Functions

| Function | Purpose | Example Result |
|----------|---------|----------------|
| `COUNT()` | Count rows | 11 |
| `AVG()` | Calculate average | 165.7 |
| `MAX()` | Find maximum value | 228 |
| `MIN()` | Find minimum value | 66 |
| `SUM()` | Add up values | 1823 |


## 🛠️ Setup: Connect to Database

**Run the next 3 cells:**


In [1]:
# Load SQL magic extension
%load_ext sql

# Fix prettytable compatibility issue
import prettytable
try:
    # Try to access DEFAULT to see if it exists
    _ = prettytable.DEFAULT
except AttributeError:
    # If it doesn't exist, add it using SINGLE_BORDER
    from prettytable import SINGLE_BORDER
    prettytable.DEFAULT = SINGLE_BORDER

# Configure SQL magic settings
%config SqlMagic.autopandas = False
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

/tmp/ipykernel_20529/3162060372.py:8: DeprecationWarning: the 'DEFAULT' constant is deprecated, use the 'TableStyle' enum instead
  _ = prettytable.DEFAULT


In [2]:
%sql sqlite:///starwars.db

In [3]:
%%sql
-- Reference: See all data with species breakdown
SELECT name, species, height FROM characters ORDER BY species, height;


name,species,height
R2-D2,Droid,96
Leia Organa,Human,150
Padmé Amidala,Human,165
Luke Skywalker,Human,172
Han Solo,Human,180
Obi-Wan Kenobi,Human,182
Mace Windu,Human,188
Darth Vader,Human,202
Ahsoka Tano,Togruta,170
Chewbacca,Wookiee,228


## 📚 Data Reference for Aggregates

**Character heights (useful for this lesson):**
- Yoda: 66cm
- R2-D2: 96cm  
- Princess Leia: 150cm
- Luke Skywalker: 172cm
- Darth Vader: 202cm
- Chewbacca: 228cm

**Species counts:**
- Human: 3 characters
- Droid: 1 character
- Wookiee: 1 character
- Yoda's species: 1 character

💡 **Use aggregates to calculate:** COUNT(*), AVG(height), MAX(height), MIN(height), SUM(height)


## 🔢 Part 1: Basic Aggregate Functions

### Query 1: Count All Characters

**Result:** Total number of rows (e.g., 11)


In [7]:
%%sql
-- Query 1: Count how many characters are in the table
SELECT COUNT(*) FROM characters

COUNT(*)
11


### Query 2: Count Non-NULL Values

**Note:** `COUNT(column)` ignores NULL values, but `COUNT(*)` counts all rows.


In [9]:
%%sql
-- Query 2: Count characters who have a height recorded
SELECT COUNT(name) FROM characters

COUNT(name)
11


### Query 3: Find Maximum Height

**Result:** Should show 228 (Chewbacca's height)


In [10]:
%%sql
-- Query 3: Find the tallest character's height
SELECT MAX(height) FROM characters

MAX(height)
228


### Query 4: Find Minimum Height

**Result:** Should show 66 (Yoda's height)


In [11]:
%%sql
-- Query 4: Find the shortest character's height
SELECT MIN(height) FROM characters

MIN(height)
66


### Query 5: Calculate Average Height

**Result:** Approximately 158 cm (average of all characters with height data)


In [12]:
%%sql
-- Query 5: Calculate average height
SELECT AVG(height) FROM characters

AVG(height)
163.54545454545453


### Query 6: Sum All Heights

Add up all character heights together:


In [13]:
%%sql
-- Query 6: Sum all heights
SELECT SUM(height) FROM characters

SUM(height)
1799


### Query 7: Use Multiple Aggregates Together

**Explanation:** `AS` creates an alias (friendly name) for the result column.


In [35]:
%%sql
-- Query 7: Get stats in one query with column aliases
SELECT
    COUNT(*) AS total_characters,
    AVG(height) AS average_height,
    MAX(height) AS tall,
    MIN(height) AS short
FROM characters

total_characters,average_height,tall,short
11,163.54545454545453,228,66


## 📊 Part 2: GROUP BY Clause

GROUP BY groups rows that have the same values and performs aggregate functions on each group.

### GROUP BY Syntax

```sql
SELECT column, AGGREGATE_FUNCTION(column)
FROM table_name
GROUP BY column;
```

### Query 8: Count Characters by Species

**Result:** Shows each species and how many characters belong to it.


In [38]:
%%sql
-- Query 8: Count characters by species
SELECT COUNT(species) FROM characters GROUP BY species

COUNT(species)
1
7
1
1
1


### Query 9: Average Height by Species

**Note:** `WHERE height IS NOT NULL` filters out characters without height data before grouping.


In [39]:
%%sql
-- Query 9: Average height by species
SELECT AVG(height) FROM characters GROUP BY species

AVG(height)
96.0
177.0
170.0
228.0
66.0


### Query 10: Count Characters by Homeworld

**Explanation:** Groups by homeworld, counts each group, then sorts by count (highest first).


In [43]:
%%sql
-- Query 10: Count and average height by species, sorted by count
SELECT COUNT(homeworld) FROM characters GROUP BY homeworld ORDER BY COUNT(homeworld) DESC

COUNT(homeworld)
2
2
1
1
1
1
1
1
1


## 🏛️ Part 3: Add Affiliation Data

Before the next examples, let's add affiliation data to track which organisations characters belong to.

**Run the next 2 cells to add and populate the affiliation column:**


In [66]:
%%sql
-- Add affiliation column
ALTER TABLE characters ADD COLUMN affiliation TEXT;

[]

In [67]:
%%sql
-- Update characters with affiliations
UPDATE characters SET affiliation = 'Rebel Alliance' WHERE name IN ('Luke Skywalker', 'Leia Organa', 'Han Solo', 'Chewbacca');
UPDATE characters SET affiliation = 'Jedi Order' WHERE name IN ('Obi-Wan Kenobi', 'Yoda', 'Ahsoka Tano');
UPDATE characters SET affiliation = 'Galactic Empire' WHERE name = 'Darth Vader';
UPDATE characters SET affiliation = 'Independent' WHERE name = 'R2-D2';
UPDATE characters SET affiliation = 'Galactic Republic' WHERE name IN ('Padmé Amidala', 'Mace Windu');

[]

### Verify Affiliation Data


In [71]:
%%sql
-- Query 13: Count characters by homeworld
SELECT name, affiliation FROM characters ORDER BY affiliation

name,affiliation
Darth Vader,Galactic Empire
Padmé Amidala,Galactic Republic
Mace Windu,Galactic Republic
R2-D2,Independent
Obi-Wan Kenobi,Jedi Order
Yoda,Jedi Order
Ahsoka Tano,Jedi Order
Luke Skywalker,Rebel Alliance
Leia Organa,Rebel Alliance
Han Solo,Rebel Alliance


### Query 11: Count by Affiliation


In [75]:
%%sql
-- Query 11: Count characters in each affiliation
SELECT AFFILIATION, COUNT(*) AS members FROM characters GROUP BY affiliation ORDER BY members 

affiliation,members
Galactic Empire,1
Independent,1
Galactic Republic,2
Jedi Order,3
Rebel Alliance,4


## 🎯 Part 4: HAVING Clause

`HAVING` filters groups (like WHERE filters rows). Use HAVING with GROUP BY to filter aggregated results.

### WHERE vs HAVING

| Clause | Filters | Used With | Example |
|--------|---------|-----------|---------|
| `WHERE` | Individual rows | Before grouping | `WHERE species = 'Human'` |
| `HAVING` | Groups | After grouping | `HAVING COUNT(*) > 2` |

### HAVING Syntax

```sql
SELECT column, AGGREGATE_FUNCTION(column)
FROM table_name
GROUP BY column
HAVING condition;
```

### Query 12: Find Species with Multiple Characters

**Result:** Only shows species that have 2+ characters.


In [112]:
%%sql
SELECT species, COUNT(*) AS population FROM characters GROUP BY species HAVING COUNT(*) >= 2

species,population
Human,7


### Query 13: Combine WHERE and HAVING

**Explanation:**
1. `WHERE species = 'Human'` - Filter to humans first
2. `GROUP BY homeworld` - Group remaining rows by planet
3. `HAVING COUNT(*) >= 2` - Only show groups with 2+ characters


In [113]:
%%sql
-- Example 2: Homeworlds with at least 2 human characters
SELECT homeworld, species FROM characters WHERE species = 'Human' GROUP BY homeworld HAVING COUNT(species) >= 2

homeworld,species
Tatooine,Human


## �� Part 5: COUNT DISTINCT

`COUNT(DISTINCT column)` counts unique values, ignoring duplicates.

### Query 14: Count Unique Species


In [115]:
%%sql
-- Exercise 1: Find maximum and minimum height
SELECT COUNT(DISTINCT species) AS species FROM characters

species
5


### Query 15: Count Unique Homeworlds


In [116]:
%%sql
-- Exercise 2: Count how many droids
SELECT COUNT(DISTINCT homeworld) AS homeworlds FROM characters

homeworlds
9


## 🎓 Practice Exercises

Now it's your turn! Complete these queries:


### Exercise 1: Find Total Height


In [117]:
%%sql
-- Exercise 3: Average height by homeworld
SELECT SUM(height) FROM characters

SUM(height)
1799


### Exercise 2: Group and Count


In [121]:
%%sql
-- Exercise 2: Count characters from each homeworld, sorted alphabetically
SELECT homeworld, COUNT(*) AS ughhhghghghghgh FROM characters GROUP BY homeworld ORDER BY homeworld

homeworld,ughhhghghghghgh
Alderaan,1
Corellia,1
Haruun Kal,1
Kashyyyk,1
Naboo,2
Shili,1
Stewjon,1
Tatooine,2
Unknown,1


### Exercise 3: Average with Grouping


In [123]:
%%sql
-- Exercise 3: Find average height by affiliation
SELECT height, affiliation, AVG(height) AS average FROM characters GROUP BY affiliation

height,affiliation,average
202,Galactic Empire,202.0
165,Galactic Republic,176.5
96,Independent,96.0
182,Jedi Order,139.33333333333334
172,Rebel Alliance,182.5


### Exercise 4: Using HAVING


In [125]:
%%sql
-- Exercise 4: Show homeworlds that have exactly 1 character
SELECT homeworld, COUNT(*) AS population FROM characters GROUP BY homeworld HAVING COUNT(*) = 1

homeworld,population
Alderaan,1
Corellia,1
Haruun Kal,1
Kashyyyk,1
Shili,1
Stewjon,1
Unknown,1


## 🎯 Challenge Problem

**Task:** Find which affiliation has the tallest average height, but only include affiliations with 2 or more members. Show the affiliation name, average height (rounded to 2 decimals), and member count, sorted by average height (tallest first).

**Requirements:**
- Use AVG(), COUNT()
- Use GROUP BY
- Use HAVING to filter groups
- Use ORDER BY
- Use ROUND(AVG(height), 2)


In [126]:
%%sql
SELECT height, affiliation, ROUND(AVG(height),2) AS average, COUNT(affiliation) AS a FROM characters GROUP BY affiliation HAVING a >=2


height,affiliation,average,a
165,Galactic Republic,176.5,2
182,Jedi Order,139.33,3
172,Rebel Alliance,182.5,4


## 🐛 Common Errors & Troubleshooting

### Error: "misuse of aggregate function"

**Problem:** Using aggregate function without GROUP BY when other columns are selected.

**Wrong:**
```sql
SELECT species, COUNT(*)
FROM characters;
```

**Correct:**
```sql
SELECT species, COUNT(*)
FROM characters
GROUP BY species;
```

**Rule:** If you SELECT a column and an aggregate, you must GROUP BY that column.

### WHERE vs HAVING Confusion

**Wrong:**
```sql
-- Can't use aggregate in WHERE
SELECT species, COUNT(*) 
FROM characters 
WHERE COUNT(*) > 2 
GROUP BY species;
```

**Correct:**
```sql
-- Use HAVING for aggregate conditions
SELECT species, COUNT(*) 
FROM characters 
GROUP BY species
HAVING COUNT(*) > 2;
```

### NULL Values in Aggregates

**Important:** 
- `COUNT(*)` counts all rows (including NULLs)
- `COUNT(column)` counts non-NULL values
- `AVG()`, `SUM()`, `MAX()`, `MIN()` ignore NULLs

### Order of Clauses

**Correct order:**
```sql
SELECT columns
FROM table
WHERE condition       -- Filter rows first
GROUP BY column       -- Then group
HAVING condition      -- Filter groups
ORDER BY column       -- Finally sort
LIMIT number;
```


## ✅ Checkpoint & Summary

### What You've Learnt

- ✅ Use COUNT(), AVG(), MAX(), MIN(), SUM()
- ✅ Group data with GROUP BY
- ✅ Filter groups with HAVING
- ✅ Understand WHERE (rows) vs HAVING (groups)
- ✅ Use COUNT(DISTINCT) for unique values
- ✅ Combine aggregates with ORDER BY and LIMIT
- ✅ Write column aliases with AS

### Key SQL Commands

| Command | Purpose | Example |
|---------|---------|---------|
| `COUNT()` | Count rows | `SELECT COUNT(*) FROM characters` |
| `AVG()` | Calculate average | `SELECT AVG(height) FROM characters` |
| `MAX()` | Find maximum | `SELECT MAX(height) FROM characters` |
| `MIN()` | Find minimum | `SELECT MIN(height) FROM characters` |
| `SUM()` | Add values | `SELECT SUM(height) FROM characters` |
| `GROUP BY` | Group rows | `GROUP BY species` |
| `HAVING` | Filter groups | `HAVING COUNT(*) > 2` |
| `AS` | Column alias | `COUNT(*) AS total` |
| `DISTINCT` | Unique values | `COUNT(DISTINCT species)` |
| `ROUND()` | Round numbers | `ROUND(AVG(height), 2)` |

## 🎉 Excellent Work!

You can now perform calculations and analyse your data! In the next lesson, you'll learn about database relationships and creating multiple related tables.

**Ready to continue?** Open `lesson5_schema.ipynb`

---

## 💾 Git Commands (for reference)

```bash
git status
git add lessons/lesson4_aggregates.ipynb
git commit -m "Completed Lesson 4: Aggregate functions and GROUP BY"
git push
```
